Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [235]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

First reload the data we generated in _notmist.ipynb_.

In [236]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [237]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [238]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [160]:
batch_size = 128
num_hidden_nodes = 1024
beta = 1e-4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables for hidden layer.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))

  # Variables for output.
  weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  logits = tf.matmul(layer_1, weights_2) + biases_2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))    

  # L2 regularization for the fully connected parameters.
  regularizers = beta*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2))
  
  # Add the regularization term to the loss.  
  loss += regularizers
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  layer_1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_prediction = tf.nn.softmax(tf.matmul(layer_1_valid, weights_2) + biases_2)

  layer_1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_prediction = tf.nn.softmax(tf.matmul(layer_1_test, weights_2) + biases_2)

In [161]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 331.585205
Minibatch accuracy: 11.7%
Validation accuracy: 29.6%
Minibatch loss at step 500: 43.177288
Minibatch accuracy: 85.2%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 42.574409
Minibatch accuracy: 78.9%
Validation accuracy: 82.9%
Minibatch loss at step 1500: 34.411674
Minibatch accuracy: 89.8%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 34.685200
Minibatch accuracy: 85.9%
Validation accuracy: 83.8%
Minibatch loss at step 2500: 32.060478
Minibatch accuracy: 87.5%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 31.416916
Minibatch accuracy: 84.4%
Validation accuracy: 84.1%
Minibatch loss at step 3500: 30.834221
Minibatch accuracy: 82.0%
Validation accuracy: 83.3%
Minibatch loss at step 4000: 30.008715
Minibatch accuracy: 85.2%
Validation accuracy: 83.3%
Minibatch loss at step 4500: 28.488045
Minibatch accuracy: 88.3%
Validation accuracy: 83.6%
Minibatch loss at step 5000: 27.791565
Minibatch accuracy: 87.5%
Valida

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [56]:
batch_size = 5
num_hidden_nodes = 1024
beta = beta = 1e-4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables for hidden layer.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))

  # Variables for output.
  weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  logits = tf.matmul(layer_1, weights_2) + biases_2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))    

  # L2 regularization for the fully connected parameters.
  regularizers = beta*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2))
  
  # Add the regularization term to the loss.  
  loss += regularizers
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  layer_1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_prediction = tf.nn.softmax(tf.matmul(layer_1_valid, weights_2) + biases_2)

  layer_1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_prediction = tf.nn.softmax(tf.matmul(layer_1_test, weights_2) + biases_2)

In [57]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 219.502502
Minibatch accuracy: 20.0%
Validation accuracy: 17.1%
Minibatch loss at step 500: 1025.138428
Minibatch accuracy: 60.0%
Validation accuracy: 55.2%
Minibatch loss at step 1000: 198.490097
Minibatch accuracy: 20.0%
Validation accuracy: 64.2%
Minibatch loss at step 1500: 212.287277
Minibatch accuracy: 80.0%
Validation accuracy: 55.5%
Minibatch loss at step 2000: 356.738647
Minibatch accuracy: 60.0%
Validation accuracy: 65.1%
Minibatch loss at step 2500: 73.952950
Minibatch accuracy: 80.0%
Validation accuracy: 62.9%
Minibatch loss at step 3000: 79.246986
Minibatch accuracy: 100.0%
Validation accuracy: 59.8%
Minibatch loss at step 3500: 3298.208252
Minibatch accuracy: 20.0%
Validation accuracy: 60.0%
Minibatch loss at step 4000: 9064.140625
Minibatch accuracy: 60.0%
Validation accuracy: 52.5%
Minibatch loss at step 4500: 411.528534
Minibatch accuracy: 20.0%
Validation accuracy: 54.0%
Minibatch loss at step 5000: 115.141296
Minibatch accuracy: 

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [78]:
batch_size = 5
num_hidden_nodes = 1024
beta = 1e-4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables for hidden layer.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes]))

  # Variables for output.
  weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)

  keep_prob = tf.placeholder(tf.float32)
  layer_1_drop = tf.nn.dropout(layer_1, keep_prob)

  #logits = tf.matmul(layer_1, weights_2) + biases_2
  logits = tf.matmul(layer_1_drop, weights_2) + biases_2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))    

  # L2 regularization for the fully connected parameters.
  regularizers = beta*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2))
  
  # Add the regularization term to the loss.  
  loss += regularizers
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  layer_1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_prediction = tf.nn.softmax(tf.matmul(layer_1_valid, weights_2) + biases_2)

  layer_1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_prediction = tf.nn.softmax(tf.matmul(layer_1_test, weights_2) + biases_2)

In [79]:
num_steps = 10000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.8}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 633.905396
Minibatch accuracy: 20.0%
Validation accuracy: 18.8%
Minibatch loss at step 500: 52523.898438
Minibatch accuracy: 60.0%
Validation accuracy: 50.9%
Minibatch loss at step 1000: 11659.060547
Minibatch accuracy: 80.0%
Validation accuracy: 52.3%
Minibatch loss at step 1500: 3375250.000000
Minibatch accuracy: 40.0%
Validation accuracy: 49.5%
Minibatch loss at step 2000: 1501795.000000
Minibatch accuracy: 80.0%
Validation accuracy: 59.2%
Minibatch loss at step 2500: 6707677.000000
Minibatch accuracy: 80.0%
Validation accuracy: 60.4%
Minibatch loss at step 3000: 3686816512.000000
Minibatch accuracy: 60.0%
Validation accuracy: 46.5%
Minibatch loss at step 3500: 30206107648.000000
Minibatch accuracy: 20.0%
Validation accuracy: 49.6%
Minibatch loss at step 4000: 185030328320.000000
Minibatch accuracy: 20.0%
Validation accuracy: 38.1%
Minibatch loss at step 4500: 200568717312.000000
Minibatch accuracy: 20.0%
Validation accuracy: 58.9%
Minibatch los

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [239]:
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables
  batch_size = 128
  beta = 1e-5

  num_hidden_nodes_1 = 1024
  num_hidden_nodes_2 = 1024
  num_hidden_nodes_3 = 512
  num_hidden_nodes_4 = 256
  num_hidden_nodes_5 = 128

  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_nodes_1], stddev=math.sqrt(2.0/(image_size*image_size))))
  weights_2 = tf.Variable(tf.truncated_normal([num_hidden_nodes_1, num_hidden_nodes_2], stddev=math.sqrt(2.0/num_hidden_nodes_1)))
  weights_3 = tf.Variable(tf.truncated_normal([num_hidden_nodes_2, num_hidden_nodes_3], stddev=math.sqrt(2.0/num_hidden_nodes_2)))
  weights_4 = tf.Variable(tf.truncated_normal([num_hidden_nodes_3, num_hidden_nodes_4], stddev=math.sqrt(2.0/num_hidden_nodes_3)))
  weights_5 = tf.Variable(tf.truncated_normal([num_hidden_nodes_4, num_hidden_nodes_5], stddev=math.sqrt(2.0/num_hidden_nodes_4)))
  weights_6 = tf.Variable(tf.truncated_normal([num_hidden_nodes_5, num_labels], stddev=math.sqrt(2.0/num_hidden_nodes_5)))

  biases_1 = tf.Variable(tf.zeros([num_hidden_nodes_1]))
  biases_2 = tf.Variable(tf.zeros([num_hidden_nodes_2]))
  biases_3 = tf.Variable(tf.zeros([num_hidden_nodes_3]))
  biases_4 = tf.Variable(tf.zeros([num_hidden_nodes_4]))
  biases_5 = tf.Variable(tf.zeros([num_hidden_nodes_5]))
  biases_6 = tf.Variable(tf.zeros([num_labels]))

  keep_prob_1 = tf.placeholder(tf.float32)
  keep_prob_2 = tf.placeholder(tf.float32)
  keep_prob_3 = tf.placeholder(tf.float32)
  keep_prob_4 = tf.placeholder(tf.float32)
  keep_prob_5 = tf.placeholder(tf.float32)

  # Training computation.
  layer_1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1), keep_prob_1)
  layer_2 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer_1, weights_2) + biases_2), keep_prob_2)
  layer_3 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer_2, weights_3) + biases_3), keep_prob_3)
  layer_4 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer_3, weights_4) + biases_4), keep_prob_4)
  layer_5 = tf.nn.dropout(tf.nn.relu(tf.matmul(layer_4, weights_5) + biases_5), keep_prob_5)
  logits = tf.matmul(layer_5, weights_6) + biases_6
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))    

  # L2 regularization for the fully connected parameters.
  regularizers = beta*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2) + tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(biases_3) + tf.nn.l2_loss(weights_4) + tf.nn.l2_loss(biases_4) + tf.nn.l2_loss(weights_5) + tf.nn.l2_loss(biases_5) + tf.nn.l2_loss(weights_6) + tf.nn.l2_loss(biases_6))
  
  # Add the regularization term to the loss.  
  loss += regularizers
    
  # Optimizer
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(1.0, global_step, 1, 0.99995)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_layer_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1)
  valid_layer_2 = tf.nn.relu(tf.matmul(valid_layer_1, weights_2) + biases_2) 
  valid_layer_3 = tf.nn.relu(tf.matmul(valid_layer_2, weights_3) + biases_3) 
  valid_layer_4 = tf.nn.relu(tf.matmul(valid_layer_3, weights_4) + biases_4)
  valid_layer_5 = tf.nn.relu(tf.matmul(valid_layer_4, weights_5) + biases_5)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_layer_5, weights_6) + biases_6)

  test_layer_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1)
  test_layer_2 = tf.nn.relu(tf.matmul(test_layer_1, weights_2) + biases_2)  
  test_layer_3 = tf.nn.relu(tf.matmul(test_layer_2, weights_3) + biases_3) 
  test_layer_4 = tf.nn.relu(tf.matmul(test_layer_3, weights_4) + biases_4)
  test_layer_5 = tf.nn.relu(tf.matmul(test_layer_4, weights_5) + biases_5)
  test_prediction = tf.nn.softmax(tf.matmul(test_layer_5, weights_6) + biases_6)

In [240]:
num_steps = 50000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob_1: 0.8, keep_prob_2: 0.85, keep_prob_3: 0.9, keep_prob_4: 0.95, keep_prob_5: 1.0}
    #feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.414189
Minibatch accuracy: 7.0%
Validation accuracy: 16.0%
Minibatch loss at step 500: 0.432777
Minibatch accuracy: 87.5%
Validation accuracy: 84.9%
Minibatch loss at step 1000: 0.499278
Minibatch accuracy: 85.9%
Validation accuracy: 86.0%
Minibatch loss at step 1500: 0.368073
Minibatch accuracy: 90.6%
Validation accuracy: 86.5%
Minibatch loss at step 2000: 0.347550
Minibatch accuracy: 93.0%
Validation accuracy: 87.7%
Minibatch loss at step 2500: 0.390058
Minibatch accuracy: 89.8%
Validation accuracy: 87.6%
Minibatch loss at step 3000: 0.454132
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 3500: 0.409318
Minibatch accuracy: 87.5%
Validation accuracy: 88.0%
Minibatch loss at step 4000: 0.402129
Minibatch accuracy: 90.6%
Validation accuracy: 88.8%
Minibatch loss at step 4500: 0.390973
Minibatch accuracy: 90.6%
Validation accuracy: 88.8%
Minibatch loss at step 5000: 0.448876
Minibatch accuracy: 89.8%
Validation accuracy